In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2
from joblib import Parallel, delayed
import os
from tqdm import tqdm

In [ ]:
def load_images(file):
    return cv2.imreadmulti(file, flags=cv2.IMREAD_GRAYSCALE)[1]

In [ ]:
def process_images(images):
    def get_mask(image, back):
        return np.abs(image - back) / np.max(image) > 0.1

    back = np.mean(images, axis=0)
    masks = Parallel(n_jobs=8)(delayed(get_mask)(image,back) for image in images)
    masks = np.array(masks)
    masks = masks.astype(np.uint8) * 255
    return masks

In [ ]:
def get_contours(masks, frame):
    def clean_frame(mask):
        image = mask.copy()
        image = cv2.medianBlur(image, 3)
        
        kernel = np.ones((3,3),np.uint8)
        image = cv2.dilate(image, kernel, iterations=9)

        image = cv2.medianBlur(image, 17)

        image = cv2.equalizeHist(image)

        contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        image = np.zeros(image.shape)
        bb = []
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                cv2.drawContours(image, [contour], 0, (255), -1)
                x, y, w, h = cv2.boundingRect(contour)
                bb.append((x, y, w, h))
                cv2.rectangle(image, (x, y), (x+w, y+h), (255), 2)

    
        kernel = np.ones((100,100),np.uint8)
        image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
        image = np.array(image, dtype=np.uint8)
        return image

    def get_boundaries(mask):
        boundaries = np.zeros(mask.shape)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                cv2.drawContours(boundaries, [contour], 0, (255), 5)
        return np.array(boundaries, dtype=np.uint8)

    def get_circles(mask):
        return cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1, minDist=500, param1=50, param2=20, minRadius=270, maxRadius=350)

    def get_bounding_boxes(mask):
        circles = get_circles(mask)
        bb = []
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0,:]:
                x, y, r = circle
                x = int(x)
                y = int(y)
                r = int(r)
                if x-r < 0 or y-r < 0 or x+r > mask.shape[1] or y+r > mask.shape[0]:
                    continue
                bb.append((x-r, y-r, 2*r, 2*r))
        return bb
    
    mask = masks[frame-1]
    mask = clean_frame(mask)
    mask = get_boundaries(mask)
    bb   = get_bounding_boxes(mask)

    plt.imshow(mask)
    plt.show()
    return mask, bb       

In [ ]:
def file_to_droplets(file):
    images = load_images(file)

    print(f'Processing {file} with {len(images)} frames')

    masks  = process_images(images)

    print(f'Extracting droplets from {file}')
    
    for frame in tqdm(range(1, 31)):
        droplets = list()

        _ , bb = get_contours(masks, frame)

        for x, y, w, h in bb:
            droplets.append(masks[frame-1][y:y+h, x:x+w])

        file_name = file.split("/")[-1].split(".")[0]+"-"+str(frame).zfill(2)

        file_name = f'./renaud_data/droplets/{file_name}'
        cv2.imwritemulti(file_name+'.tif', droplets)
        
        with open(file_name+'.json', 'w') as f:
            json.dump(bb, f)
            
    print(f'Finished {file}')

file_list = os.listdir('./renaud_data/sequences')
file_list = [file for file in file_list if file.endswith('.tif')]

for file in file_list:
    file_to_droplets(f'./renaud_data/sequences/{file}')


In [ ]:
def load_json(file):
    with open(file) as f:
        return json.load(f)

def load_images(file):
    return cv2.imreadmulti(file, flags=cv2.IMREAD_GRAYSCALE)[1]

In [ ]:
def process_images(images, back):
    def get_mask(image, back):
        return np.abs(image - back) / np.max(image) > 0.1

    masks = Parallel(n_jobs=8)(delayed(get_mask)(image, back) for image in images)
    masks = np.array(masks)
    masks = masks.astype(np.uint8) * 255
    return masks

def get_contours(masks, frame):
    def clean_frame(mask):
        image = mask.copy()
        image = cv2.medianBlur(image, 3)
        
        kernel = np.ones((3,3),np.uint8)
        image = cv2.dilate(image, kernel, iterations=9)

        image = cv2.medianBlur(image, 17)

        contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        image = np.zeros(image.shape)
        bb = []
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                cv2.drawContours(image, [contour], 0, (255), -1)
                x, y, w, h = cv2.boundingRect(contour)
                bb.append((x, y, w, h))
                cv2.rectangle(image, (x, y), (x+w, y+h), (255), 2)

    
        kernel = np.ones((100,100),np.uint8)
        image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
        image = np.array(image, dtype=np.uint8)
        return image

    def get_boundaries(mask):
        boundaries = np.zeros(mask.shape)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                cv2.drawContours(boundaries, [contour], 0, (255), 5)
        return np.array(boundaries, dtype=np.uint8)

    def get_circles(mask):
        return cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1, minDist=500, param1=50, param2=20, minRadius=270, maxRadius=350)

    def get_bounding_boxes(mask):
        circles = get_circles(mask)
        bb = []
        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0,:]:
                x, y, r = circle
                x = int(x)
                y = int(y)
                r = int(r)
                if x-r < 0 or y-r < 0 or x+r > mask.shape[1] or y+r > mask.shape[0]:
                    continue
                bb.append((x-r, y-r, 2*r, 2*r))
        return bb
    
    mask = masks[frame-1]
    mask = clean_frame(mask)
    mask = get_boundaries(mask)
    bb   = get_bounding_boxes(mask)
    return mask, bb

def clean_frame(mask):
    image = mask.copy()
    image = cv2.medianBlur(image, 3)
    
    kernel = np.ones((3,3),np.uint8)
    image = cv2.dilate(image, kernel, iterations=9)

    image = cv2.medianBlur(image, 17)

    contours, _ = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    image = np.zeros(image.shape)
    bb = []
    for contour in contours:
        if cv2.contourArea(contour) > 300:
            cv2.drawContours(image, [contour], 0, (255), -1)
            x, y, w, h = cv2.boundingRect(contour)
            bb.append((x, y, w, h))
            cv2.rectangle(image, (x, y), (x+w, y+h), (255), 2)


    kernel = np.ones((100,100),np.uint8)
    image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    image = np.array(image, dtype=np.uint8)
    return image

def get_circles_bis(mask):
    return cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, dp=1, minDist=20, param1=50, param2=20, minRadius=10, maxRadius=30)
def get_circles_ter(mask):
    return cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT_ALT, dp=1.5, minDist=20, param1=500, param2=0.01, minRadius=10, maxRadius=30)

def search_cell(image):
    
    cells_found = []
    #load mask of the current box
    clean = clean_frame(image)
    cleaned_mask = image.copy()
    
    contours, _ = cv2.findContours(clean, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #check larger contour by area
    max_area = 0
    max_contour = None
    for contour in contours:
        if cv2.contourArea(contour) > max_area:
            max_area = cv2.contourArea(contour)
            max_contour = contour

    ellipse = cv2.fitEllipse(max_contour,)
    black = np.zeros_like(cleaned_mask)
    cv2.ellipse(black, ellipse, (255, 255, 255), -1)
    contour_ellipse, _ = cv2.findContours(black, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)



    cleaned_mask = image.copy()
    for i in range(cleaned_mask.shape[0]):
        for j in range(cleaned_mask.shape[1]):
            #if i,j is in the contour
            if not cv2.pointPolygonTest(contour_ellipse[0], (i,j), False) > 0:
                cleaned_mask[i,j] = 0
    
    """ 
    circles = cv2.HoughCircles(cleaned_mask, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=250, maxRadius=400)
    mask = np.zeros_like(cleaned_mask)
    if circles is not None:
        circle = np.array(circles[0][0], dtype=int)
        mask = cv2.circle(mask, (circle[0],circle[1]), circle[2], (255, 255, 255), -1)
        cleaned_mask = cv2.bitwise_and(cleaned_mask, mask)
    """
    
    circle_mask = cleaned_mask
    
    cells_found = get_circles_bis(circle_mask)[0]
    return cells_found

In [ ]:
filepath = "renaud_data/droplets/"
files = os.listdir(filepath)
files = [file for file in files if file.endswith('.tif')]
files.sort()

sequence_statistics = []

for file in files:
    frame_stats = []
    print(filepath+file)
    tif = load_images(filepath+file)
    print(len(tif))

    for image in tif:
        cells_found = search_cell(image)
        image_color = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        for circle in cells_found:
            x, y, r = circle
            x = int(x)
            y = int(y)
            r = int(r)
            cv2.circle(image_color, (x, y), r, (255, 255, 0), 2)
            cv2.circle(image_color, (x, y), 2, (255, 0, 255), 3)
        plt.imshow(image_color, cmap='gray')
        plt.show()